# File and libraries

In [6]:
import pandas as pd
import numpy as np


filepath=r'google_merch_store_raw_merge.csv'

sample_filepath=r'google_merch_store_ga4_merch_flattened_raw_000000000000.csv'



file_tag = "ga4_merch_store"

# DSLabs functions

In [7]:
%run "dslabs_functions.py"


# data functions

In [8]:
%run "data_functions.py"


data_functions lodaded


# Load

In [9]:
test_data=True
# test_data=False

if test_data==True:
    # Specify the chunk size (e.g., load 1000 lines at a time)
    chunk_size = 50000

    # Initialize an empty list to store the chunks
    chunks = []

    # Load the first chunk (or any number of chunks you want)
    for chunk in pd.read_csv(filepath, chunksize=chunk_size, parse_dates=["event_date"],date_format="%Y-%m-%d"):
        chunks.append(chunk)
        # Break after the first chunk to load only a sample
        break

    # Concatenate the chunks into a single DataFrame (optional)
    data = pd.concat(chunks)
    

else:
    data=pd.read_csv(filepath)
 
    data=data.sample(frac=0.05, replace=False)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 388298 entries, 6562436 to 5140486
Data columns (total 48 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   event_date                       388298 non-null  object 
 1   session_id                       388298 non-null  int64  
 2   user_pseudo_id                   388298 non-null  float64
 3   event_name                       388298 non-null  object 
 4   event_timestamp                  388298 non-null  int64  
 5   page_location                    388298 non-null  object 
 6   page_title                       387246 non-null  object 
 7   device_category                  388298 non-null  object 
 8   device_mobile_brand_name         388298 non-null  object 
 9   device_mobile_model_name         388298 non-null  object 
 10  device_mobile_marketing_name     388298 non-null  object 
 11  device_operating_system          388298 non-null  object 
 12  

In [10]:
data.shape

(388298, 48)

In [11]:
summary5 = data.describe(include="all")

summary5

,event_date,session_id,user_pseudo_id,event_name,event_timestamp,page_location,page_title,device_category,device_mobile_brand_name,device_mobile_model_name,...,item_id,item_name,item_brand,item_variant,item_category,price,quantity,item_revenue,item_list_index,promotion_name
count,388298,3.882980e+05,3.882980e+05,388298,3.882980e+05,388298,387246,388298,388298,388298,...,199404,199404,199289,198182,198577,191841.000000,7506.000000,199404.000000,194517.000000,198623
unique,92,NaN,NaN,17,NaN,1171,477,3,8,10,...,1168,427,6,44,75,NaN,NaN,NaN,NaN,6
top,2020-12-08,NaN,NaN,view_item,NaN,https://shop.googlemerchandisestore.com/Google...,Home,desktop,Apple,Chrome,...,(not set),(not set),(not set),(not set),Home/Apparel/Men's / Unisex/,NaN,NaN,NaN,NaN,(not set)
freq,8607,NaN,NaN,144594,NaN,36225,62716,225626,165077,107398,...,6626,7563,184762,192338,33172,NaN,NaN,NaN,NaN,188690
mean,NaN,4.991090e+09,2.601519e+08,NaN,1.607971e+15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,26.545462,2.531042,0.088474,7.022903,NaN
std,NaN,2.880111e+09,1.178007e+09,NaN,2.135877e+12,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,21.776126,91.110014,2.564276,4.004193,NaN
min,NaN,1.205000e+03,1.000631e+06,NaN,1.604189e+15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,-1.000000,0.000000,1.000000,NaN
25%,NaN,2.502050e+09,5.803240e+06,NaN,1.606370e+15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,13.000000,1.000000,0.000000,4.000000,NaN
50%,NaN,4.996862e+09,2.299635e+07,NaN,1.607688e+15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,22.000000,1.000000,0.000000,7.000000,NaN
75%,NaN,7.494179e+09,6.158408e+07,NaN,1.609795e+15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,30.000000,1.000000,0.000000,10.000000,NaN


# class target column creation


we want to classify if that hit is from a returning or new user so we need to group the ga_session_number by 1 or more than 1 session. in this case, new user (ga_session_number = 1) will be 0 and returning user will be more than 1 (ga_session_number > 1)

In [12]:
data['returning_user'] = data['ga_session_number'].apply(lambda x: 0 if x == 1 else 1)


# datetime columns

In [13]:
data['event_timestamp'] = pd.to_datetime(data['event_timestamp'], infer_datetime_format=True)


# sort values by session id to get journey picture and useful for partitioned fillna methods
data = data.sort_values(by=['session_id', 'event_timestamp'], ascending=True)

# column drop

In [14]:
data=data.drop(['debug_mode','device_is_limited_ad_tracking','device_mobile_marketing_name','geo_metro','traffic_source_name'],axis=1)

# row drop (from values)

## specific event_names



In [15]:
data['event_name'].value_counts()

event_name
view_item              144594
page_view               67747
user_engagement         52716
add_to_cart             33457
scroll                  24505
session_start           17793
select_item             16405
first_visit             12820
view_promotion           9678
begin_checkout           4257
view_search_results      1318
add_shipping_info         963
purchase                  775
add_payment_info          695
select_promotion          483
click                      68
view_item_list             24
Name: count, dtype: int64

### event_name insights
We want to classify a user by its interactions with the website so we want to exclude some actions that may also be biased by incorrect ga4 tracking namely:
- session_start
- first_visit
- click (low event count)
- view_item_list (may not be triggered by user interaction)

In [16]:
# List of values to drop
events_to_drop = ['session_start', 'first_visit','click','view_item_list']

# drop events from list
data = data[~data['event_name'].isin(events_to_drop)]


data['event_name'].value_counts()

event_name
view_item              144594
page_view               67747
user_engagement         52716
add_to_cart             33457
scroll                  24505
select_item             16405
view_promotion           9678
begin_checkout           4257
view_search_results      1318
add_shipping_info         963
purchase                  775
add_payment_info          695
select_promotion          483
Name: count, dtype: int64

# replace (not set) and Other with null

we will handle these later but these are actually null values

In [17]:
data.replace('(not set)', np.nan, inplace=True)

data.replace('<Other>', np.nan, inplace=True)


# geo columns

## move subcontinent

In [18]:
# Move 'geo_sub_continent' from index 21 to index 18 after continent

geo_sub_continent = data.pop('geo_sub_continent')  # Remove the column
data.insert(18, 'geo_sub_continent', geo_sub_continent)  # Insert at index 18


# Display the reordered DataFrame
print("\nReordered DataFrame:")
data.info()


Reordered DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 357593 entries, 3665809 to 2275775
Data columns (total 44 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   event_date                       357593 non-null  object        
 1   session_id                       357593 non-null  int64         
 2   user_pseudo_id                   357593 non-null  float64       
 3   event_name                       357593 non-null  object        
 4   event_timestamp                  357593 non-null  datetime64[ns]
 5   page_location                    357593 non-null  object        
 6   page_title                       356871 non-null  object        
 7   device_category                  357593 non-null  object        
 8   device_mobile_brand_name         330123 non-null  object        
 9   device_mobile_model_name         283991 non-null  object        
 10  device_operating_sys

## geo_continent

In [19]:
data = fill_nulls_based_on_top_value_multiple_columns(data, 'geo_continent', ['device_category'])

data['geo_continent'].value_counts()

geo_continent
Americas    202262
Asia         81969
Europe       66004
Oceania       3867
Africa        3491
Name: count, dtype: int64

## geo_sub_continent

In [20]:
data = fill_nulls_based_on_top_value_multiple_columns(data, 'geo_sub_continent', ['device_category','geo_continent'])

data['geo_sub_continent'].value_counts()

geo_sub_continent
Northern America    187921
Southern Asia        35522
Eastern Asia         23143
Western Europe       21306
Northern Europe      18477
Southern Europe      16502
Southeast Asia       14077
Eastern Europe        9719
South America         9321
Western Asia          8974
Central America       4090
Australasia           3867
Northern Africa       1788
Caribbean              930
Western Africa         647
Southern Africa        490
Eastern Africa         463
Central Asia           253
Middle Africa          103
Name: count, dtype: int64

## geo_country

In [21]:
data = fill_nulls_based_on_top_value_multiple_columns(data, 'geo_country', ['device_category','geo_continent'])

data['geo_country'].value_counts()

geo_country
United States        160787
India                 33112
Canada                27448
United Kingdom        11488
Spain                  6865
                      ...  
Ghana                    47
Trinidad & Tobago        46
Oman                     41
Honduras                 34
Kosovo                   29
Name: count, Length: 108, dtype: int64

## geo_region and geo_city

In [22]:
data = fill_nulls_based_on_top_value_multiple_columns(data, 'geo_region', ['device_category','geo_continent','geo_country'])

data['geo_region'].value_counts()

geo_region
California                 36990
Texas                      12394
Ontario                    11490
New York                   10739
Virginia                   10467
                           ...  
Guayas                        30
Central Bohemian Region       29
Community of Madrid           29
Bratislava Region             28
Kocaeli                       28
Name: count, Length: 366, dtype: int64

In [23]:
null_countries_df=data.groupby(['geo_country']).agg(
    null_count=('geo_region', lambda x: x.isna().sum())
).sort_values('null_count',ascending=False).reset_index()


countries_with_nulls = null_countries_df[null_countries_df['null_count'] > 0]['geo_country'].tolist()

print(countries_with_nulls)
# will retrieve the most populated cities of these countries to use as fill method

['Singapore', 'Hong Kong', 'Peru', 'Dominican Republic', 'Iraq', 'Serbia', 'Uruguay', 'Palestine', 'Armenia', 'North Macedonia', 'Bahrain', 'Cyprus', 'Panama', 'Albania', 'Venezuela', 'Luxembourg', 'Bolivia', 'Jamaica', 'Azerbaijan', 'Paraguay', 'Macao', 'Malta', 'Trinidad & Tobago', 'Honduras', 'Kosovo', 'Puerto Rico', 'Slovakia', 'Bangladesh', 'New Zealand', 'Costa Rica']


In [24]:
most_populated_cities = {
    'Singapore': 'Singapore',
    'Hong Kong': 'Hong Kong',
    'Peru': 'Lima',
    'Dominican Republic': 'Santo Domingo',
    'Palestine': 'Gaza',
    'Iraq': 'Baghdad',
    'Serbia': 'Belgrade',
    'Uruguay': 'Montevideo',
    'North Macedonia': 'Skopje',
    'Cyprus': 'Nicosia',
    'Bahrain': 'Manama',
    'Armenia': 'Yerevan',
    'Panama': 'Panama City',
    'Azerbaijan': 'Baku',
    'Venezuela': 'Caracas',
    'Albania': 'Tirana',
    'Luxembourg': 'Luxembourg',
    'Jamaica': 'Kingston',
    'Macao': 'Macau',
    'Bolivia': 'Santa Cruz de la Sierra',
    'Trinidad & Tobago': 'Chaguanas',
    'Honduras': 'Tegucigalpa',
    'Paraguay': 'Asuncion',
    'Morocco': 'Casablanca',
    'Kosovo': 'Pristina',
    'Slovakia': 'Bratislava',
    'Malta': 'Birkirkara',
    'New Zealand': 'Auckland',
    'Bulgaria': 'Sofia',
    'Bangladesh': 'Dhaka',
    'Nigeria': 'Lagos'
}



data['geo_region'] = data['geo_region'].fillna(data['geo_country'].map(most_populated_cities))
data['geo_city'] = data['geo_city'].fillna(data['geo_country'].map(most_populated_cities))

In [25]:
null_regions_df=data.groupby(['geo_region']).agg(
    null_count=('geo_region', lambda x: x.isna().sum())
).sort_values('null_count',ascending=False).reset_index()

null_regions_df


,geo_region,null_count
0,Abruzzo,0
1,Porto District,0
2,Podkarpackie Voivodeship,0
3,Piedmont,0
4,Pichincha,0
...,...,...
388,Greater Accra Region,0
389,Grand Est,0
390,Giza Governorate,0
391,Georgia,0


In [26]:
null_region_cities_df=data.groupby(['geo_region']).agg(
    null_count=('geo_city', lambda x: x.isna().sum())
).sort_values('null_count',ascending=False).reset_index()


region_cities_with_nulls = null_region_cities_df[null_region_cities_df['null_count'] > 0]['geo_region'].tolist()


region_cities_with_nulls

['California',
 'Florida',
 'England',
 'Texas',
 'New Jersey',
 'Pennsylvania',
 'New York',
 'Michigan',
 'Ohio',
 'Taipei City',
 'Ontario',
 'Georgia',
 'Illinois',
 'Massachusetts',
 'Virginia',
 'North Carolina',
 'Quebec',
 'Maryland',
 'Connecticut',
 'Ile-de-France',
 'Wisconsin',
 'Tokyo',
 'Missouri',
 'Maharashtra',
 'Indiana',
 'Washington',
 'Colorado',
 'British Columbia',
 'Utah',
 'Catalonia',
 'South Carolina',
 'Minnesota',
 'State of Sao Paulo',
 'Alberta',
 'Bavaria',
 'Tennessee',
 'Lombardy',
 'New Taipei City',
 'Moscow',
 'Alabama',
 'Istanbul',
 'Zhejiang',
 'Oregon',
 'Louisiana',
 'North Rhine-Westphalia',
 'Zurich',
 'Baden-Wurttemberg',
 'Mexico City',
 'Flanders',
 'Iowa',
 'Kentucky',
 'Mississippi',
 'Taiwan Province',
 'Arkansas',
 'Arizona',
 'Auvergne-Rhone-Alpes',
 'Metro Manila',
 'Lisbon',
 'Oklahoma',
 'North Holland',
 'Attica',
 'Kansas',
 'Jakarta',
 'Stockholm County',
 'Bogota',
 'South Holland',
 'County Dublin',
 'Selangor',
 'Ho Chi Minh 

In [27]:
most_populated_cities_by_region = {
    'Taipei City': 'Taipei',
    'New Taipei City': 'New Taipei',
    'Taichung City': 'Taichung',
    'Kaohsiung City': 'Kaohsiung',
    'West Virginia': 'Charleston',
    'Delaware': 'Wilmington',
    'State of Mexico': 'Ecatepec',
    'Buenos Aires Province': 'La Plata',
    'Montana': 'Billings',
    'Wallonia': 'Liège',
    'Alaska': 'Anchorage',
    'Grand Est': 'Strasbourg',
    'Henan': 'Zhengzhou',
    'Hawalli Governorate': 'Hawalli',
    'Gelderland': 'Arnhem',
    'Algiers Province': 'Algiers',
    'North Dakota': 'Fargo',
    'Vermont': 'Burlington',
    'Idaho': 'Boise',
    'Canary Islands': 'Las Palmas',
    'Castile-La Mancha': 'Toledo',
    'Lublin Voivodeship': 'Lublin',
    'Calabarzon': 'Cavite City',
    'Saxony': 'Dresden',
    'Castile and Leon': 'Valladolid',
    'State of Bahia': 'Salvador',
    'Limburg': 'Hasselt',
    'Tainan City': 'Tainan',
    'Moscow Oblast': 'Khimki',
    'Chiba': 'Chiba',
    'Prince Edward Island': 'Charlottetown',
    'Overijssel': 'Zwolle',
    'Haifa District': 'Haifa',
    'Liguria': 'Genoa',
    'Northern Ireland': 'Belfast',
    'Rhineland-Palatinate': 'Mainz',
    'Sverdlovsk Oblast': 'Yekaterinburg',
    'Region of Southern Denmark': 'Odense',
    'Decentralized Administration of Peloponnese, Western Greece and the Ionian': 'Patras',
    'Amman Governorate': 'Amman',
    'Silesian Voivodeship': 'Katowice',
    'Asturias': 'Oviedo',
    'Galicia': 'Santiago de Compostela',
    'Hokkaido': 'Sapporo',
    'Quintana Roo': 'Cancún',
    'Centre-Val de Loire': 'Orléans',
    'Tamaulipas': 'Reynosa',
    'State of Santa Catarina': 'Florianópolis',
    'Saitama': 'Saitama',
    'Sicily': 'Palermo',
    'Gyeongsangnam-do': 'Changwon',
    'St. Gallen': 'St. Gallen',
    'Drenthe': 'Assen',
    'State of Ceara': 'Fortaleza',
    'Marche': 'Ancona',
    'West Pomeranian Voivodeship': 'Szczecin',
    'Friuli-Venezia Giulia': 'Trieste',
    'Decentralized Administration of Attica': 'Athens',
    'Viken': 'Drammen',
    'Lower Austria': 'St. Pölten',
    'State of Pernambuco': 'Recife',
    'Veracruz': 'Veracruz',
    'Flanders': 'Antwerp',
    'Kuyavian-Pomeranian Voivodeship': 'Bydgoszcz',
    'Krasnodar Krai': 'Krasnodar',
    'Central Bohemian Region': 'Prague',
    'State of Espirito Santo': 'Vitória',
    'Abruzzo': 'Pescara',
    'Region Zealand': 'Roskilde',
    'Maryland': 'Baltimore',
    'South District': 'Ashdod',
    'Normandy': 'Rouen',
    'Iowa': 'Des Moines',
    'Jerusalem District': 'Jerusalem',
    'Wisconsin': 'Milwaukee',
    'Gyeongsangbuk-do': 'Gyeongju',
    'Puebla': 'Puebla',
    'Friesland': 'Leeuwarden',
    'Bourgogne-Franche-Comte': 'Dijon',
    'Chungcheongbuk-do': 'Cheongju',
    'Sonora': 'Hermosillo',
    'Penang': 'George Town',
    'Federation of Bosnia and Herzegovina': 'Sarajevo',
    'Trentino-South Tyrol': 'Bolzano',
    'Cordoba': 'Córdoba',
    'Hyogo': 'Kobe',
    'Brandenburg': 'Potsdam',
    'Baden-Wurttemberg': 'Stuttgart',
    'Setubal': 'Setúbal',
    'Guanajuato': 'Guanajuato',
    'Kocaeli': 'Izmit',
    'Groningen': 'Groningen',
    'Muscat Governorate': 'Muscat',
    'Giza Governorate': 'Giza',
    'Arkansas': 'Little Rock',
    'Community of Madrid': 'Madrid',
    'Skane County': 'Malmö',
    'South Moravian Region': 'Brno',
    'Podkarpackie Voivodeship': 'Rzeszów',
    'Minnesota': 'Minneapolis',
    'Utrecht': 'Utrecht',
    'North Rhine-Westphalia': 'Cologne',
    'Indiana': 'Indianapolis',
    'Louisiana': 'New Orleans',
    'Odisha': 'Bhubaneswar',
    'Newfoundland and Labrador': 'St. John\'s',
    'Canton of Bern': 'Bern',
    'Bihar': 'Patna',
    'Utah': 'Salt Lake City',
    'Lower Saxony': 'Hanover',
    'Guayas': 'Guayaquil',
    'Gangwon-do': 'Gangneung',
    'Brittany': 'Rennes',
    'Mississippi': 'Jackson',
    'Pays de la Loire': 'Nantes',
    'Nuevo Leon': 'Monterrey',
    'New Hampshire': 'Concord',
    'South Dakota': 'Sioux Falls',
    'Chhattisgarh': 'Raipur',
    'Uttarakhand': 'Dehradun'
}

data['geo_city'] = data['geo_city'].fillna(data['geo_region'].map(most_populated_cities_by_region))


In [28]:
data = fill_nulls_based_on_top_value_multiple_columns(data, 'geo_city', ['device_category','geo_continent','geo_country','geo_region'])


# device columns

- for many cases we assumed devices, brands and os versions of 2021 as fill method
- since device market share may change by region or country we also used these to consider for mode null fill

## device mobile brand

In [29]:

# Fill 'device_mobile_brand_name' with 'PC' where the conditions are met
data.loc[(data['device_operating_system'] == 'Windows') & (data['device_category'] == 'desktop'), 'device_mobile_brand_name'] = 'PC'
data.loc[(data['device_operating_system'] == 'Web') & (data['device_category'] == 'desktop') & ((data['device_mobile_model_name'].isin(['Chrome','Edge','Firefox']))), 'device_mobile_brand_name'] = 'PC'
data.loc[(data['device_mobile_brand_name'] == 'Microsoft') & (data['device_category'] == 'desktop'), 'device_mobile_brand_name'] = 'PC'



data = fill_nulls_based_on_top_value_multiple_columns(data, 'device_mobile_brand_name', ['device_category','device_operating_system','geo_country'])

data['device_mobile_brand_name'].value_counts()

device_mobile_brand_name
Apple        165506
PC           114530
Samsung       35663
Google        27965
Xiaomi         8470
Huawei         4972
Mozilla         273
Microsoft        41
Name: count, dtype: int64

## device_mobile_model_name 

In [30]:


data.loc[(data['device_mobile_brand_name'] == 'Samsung'), 'device_mobile_model_name'] = 'Galaxy S21'

data.loc[(data['device_mobile_brand_name'] == 'Xiaomi'), 'device_mobile_model_name'] = 'Mi 11'

data.loc[(data['device_mobile_brand_name'] == 'Huawei'), 'device_mobile_model_name'] = 'P50'

data.loc[(data['device_mobile_brand_name'] == 'Apple')& (data['device_category'] == 'desktop'), 'device_mobile_model_name'] = 'Macintosh'

data.loc[(data['device_mobile_brand_name'] == 'PC')& (data['device_mobile_model_name'] == 'Chrome'), 'device_mobile_model_name'] = 'PC'



data = fill_nulls_based_on_top_value_multiple_columns(data, 'device_mobile_model_name', ['device_category','device_mobile_brand_name','device_operating_system','geo_country'])


data['device_mobile_model_name'].value_counts()

device_mobile_model_name
PC            102499
iPhone         87769
Macintosh      72249
Galaxy S21     35663
ChromeBook     19647
Mi 11           8470
Edge            7724
iPad            5478
P50             4972
Firefox         4575
Pixel 4 XL      3652
Pixel 3         3056
Chrome          1018
Safari            10
Name: count, dtype: int64

## device_operating_system

In [31]:

# Chrome OS - ensure 'desktop' is correctly spelled
data.loc[(data['device_mobile_model_name'] == 'ChromeBook') & (data['device_category'] == 'desktop'), 'device_operating_system'] = 'ChromeOS'

# iOS - for iPhone and iPad
data.loc[data['device_mobile_model_name'].isin(['iPhone', 'iPad']) | (data['device_mobile_brand_name'] == 'Apple') | ((data['device_mobile_model_name'] == 'Apple') & (data['device_category'].isin(['mobile','tablet']))), 'device_operating_system'] = 'iOS'

# Android - for specified brands
android_brands = ['Xiaomi', 'Huawei', 'Samsung']
data.loc[data['device_mobile_brand_name'].isin(android_brands), 'device_operating_system'] = 'Android'
data.loc[(data['device_mobile_brand_name'] == 'Google') & (data['device_category'].isin(['mobile','tablet'])), 'device_operating_system'] = 'Android'

# macOS
data.loc[(data['device_mobile_brand_name'] == 'Apple') & (data['device_category'] == 'desktop'), 'device_operating_system'] = 'MacOS'

# Windows
data.loc[(data['device_operating_system'] == 'Web') & (data['device_category'] == 'desktop') & ((data['device_mobile_brand_name'] == 'PC')), 'device_operating_system'] = 'Windows'
data.loc[(data['device_operating_system'] == 'Web') & (data['device_category'] == 'desktop') & ((data['device_mobile_brand_name'] == 'Mozilla')), 'device_operating_system'] = 'Windows'
data.loc[(data['device_category'] == 'desktop') & ((data['device_mobile_brand_name'] == 'Microsoft')), 'device_operating_system'] = 'Windows'


data = fill_nulls_based_on_top_value_multiple_columns(data, 'device_operating_system', ['device_category','device_operating_system','geo_country'])


data.groupby(['device_category','device_operating_system', 'device_mobile_brand_name']).agg(
    unique_event_count=('event_timestamp', 'nunique')
).sort_values('unique_event_count',ascending=False).reset_index()


,device_category,device_operating_system,device_mobile_brand_name,unique_event_count
0,desktop,Windows,PC,96541
1,mobile,iOS,Apple,74150
2,desktop,MacOS,Apple,61062
3,mobile,Android,Samsung,28551
4,desktop,ChromeOS,Google,16685
5,mobile,Android,Xiaomi,7140
6,mobile,Android,Google,6206
7,tablet,iOS,Apple,4679
8,mobile,Android,Huawei,3900
9,tablet,Android,Samsung,1629


## device_operating_system_version

In [32]:
# replace all string characters and keep float values
data['device_operating_system_version'] = data['device_operating_system_version'].str.extract(r'(\d+\.\d+|\d+)')


In [33]:
# for chrome os consider same browser version
# https://chromereleases.googleblog.com/2021/
data.loc[(data['device_operating_system'] == 'ChromeOS') & (data['device_operating_system_version'].isnull()), 'device_operating_system_version'] = data['device_web_info_browser_version']


data['device_operating_system_version'].value_counts()


device_operating_system_version
10       130199
10.15     50721
14.3      15973
14.2      15532
86.0       9481
87.0       7972
9          5261
11.1       4959
7          2978
Name: count, dtype: int64

In [34]:
data = fill_nulls_based_on_top_value_multiple_columns(data, 'device_operating_system_version', ['device_category','device_operating_system','geo_country'])

data['device_operating_system_version'].value_counts()

device_operating_system_version
10       168687
10.15     67221
14.2      49050
14.3      37798
86.0      11239
87.0       8404
9          5374
11.1       4959
7          2980
Name: count, dtype: int64

## device_language

In [35]:
data.groupby(['geo_continent','geo_country','device_language']).agg(
    unique_event_count=('event_timestamp', 'nunique')
).sort_values('unique_event_count',ascending=False).reset_index()

,geo_continent,geo_country,device_language,unique_event_count
0,Americas,United States,en-us,49660
1,Asia,India,en-us,10782
2,Americas,Canada,en-us,8495
3,Americas,United States,en-gb,7855
4,Americas,United States,en,3844
...,...,...,...,...
745,Europe,Slovenia,zh,1
746,Asia,Kazakhstan,de,1
747,Asia,Kazakhstan,zh,1
748,Asia,Kuwait,de,1


In [36]:
data.groupby(['geo_continent','geo_country']).agg(
    null_device_language_count=('device_language', lambda x: x.isna().sum())
).sort_values('null_device_language_count',ascending=False).reset_index()

,geo_continent,geo_country,null_device_language_count
0,Americas,United States,74527
1,Asia,India,14836
2,Americas,Canada,13095
3,Europe,United Kingdom,5038
4,Europe,Spain,3263
...,...,...,...
103,Asia,Macao,14
104,Americas,Honduras,14
105,Europe,Malta,13
106,Americas,Trinidad & Tobago,6


In [37]:
data = fill_nulls_based_on_top_value_multiple_columns(data, 'device_language', ['geo_country'])


data['device_language'].value_counts()

device_language
en-us    296309
en-gb     20696
zh         9825
en         9647
en-ca      7074
fr         4860
es-es      4622
de         2489
ko         2071
Name: count, dtype: int64

## device_web_info_browser

In [38]:
data.loc[(data['device_web_info_browser'] == 'Android Webview'), 'device_web_info_browser'] = "Chrome"


data = fill_nulls_based_on_top_value_multiple_columns(data, 'device_web_info_browser', ['device_category','device_operating_system','geo_country'])

data['device_web_info_browser'].value_counts()

device_web_info_browser
Chrome     257013
Safari      86464
Edge         7855
Firefox      6144
Name: count, dtype: int64

## device_web_info_browser_version

In [39]:
data = fill_nulls_based_on_top_value_multiple_columns(data, 'device_web_info_browser_version', ['device_category','device_web_info_browser','geo_country'])

data['device_web_info_browser_version'].value_counts()

device_web_info_browser_version
87.0    186908
86.0     77889
14.0     66626
604       9910
13.1      8213
83.0      2690
84.0      2285
13.0      1503
82.0      1136
14.1        18
87           9
86           1
Name: count, dtype: int64

# Traffic Columns

## session counts

In [40]:
data.groupby(['traffic_source_medium','traffic_source_source']).agg(
    unique_session_count=('session_id', 'nunique')
).sort_values('unique_session_count',ascending=False).reset_index()

,traffic_source_medium,traffic_source_source,unique_session_count
0,organic,google,32573
1,(none),(direct),24485
2,referral,shop.googlemerchandisestore.com,8835
3,(data deleted),(data deleted),7218
4,cpc,google,4497
5,referral,(data deleted),2


## null counts

In [41]:
data.groupby(['traffic_source_medium']).agg(
    null_count=('traffic_source_source', lambda x: x.isna().sum())
).sort_values('null_count',ascending=False).reset_index()

,traffic_source_medium,null_count
0,referral,34151
1,organic,9270
2,(data deleted),526
3,cpc,15
4,(none),0


In [42]:
data.groupby(['traffic_source_source']).agg(
    null_count=('traffic_source_medium', lambda x: x.isna().sum())
).sort_values('null_count',ascending=False).reset_index()

# no null count on

,traffic_source_source,null_count
0,(data deleted),0
1,(direct),0
2,google,0
3,shop.googlemerchandisestore.com,0


## remove parenthesis ()

In [43]:
data['traffic_source_source'] = data['traffic_source_source'].str.replace(r'\(|\)', '', regex=True).str.strip()
data['traffic_source_medium'] = data['traffic_source_medium'].str.replace(r'\(|\)', '', regex=True).str.strip()


data.groupby(['traffic_source_medium','traffic_source_source']).agg(
    unique_session_count=('session_id', 'nunique')
).sort_values('unique_session_count',ascending=False).reset_index()

,traffic_source_medium,traffic_source_source,unique_session_count
0,organic,google,32573
1,none,direct,24485
2,referral,shop.googlemerchandisestore.com,8835
3,data deleted,data deleted,7218
4,cpc,google,4497
5,referral,data deleted,2


## persist source and medium by session_id first

Since there may be null values inside the session we want to ensure that the value remains the same across the session

In [44]:
# Step 1: Fill null values with the last valid observation for the relevant session
data['traffic_source_source'] = data.groupby('session_id')['traffic_source_source'].fillna(method='ffill')
data['traffic_source_medium'] = data.groupby('session_id')['traffic_source_medium'].fillna(method='ffill')

# # Step 2: Only keep the values for rows where entrances = 1
# data.loc[data['entrances'] != 1, ['traffic_source_source', 'traffic_source_medium']] = None


data.groupby(['traffic_source_medium']).agg(
    null_count=('traffic_source_source', lambda x: x.isna().sum())
).sort_values('null_count',ascending=False).reset_index()

,traffic_source_medium,null_count
0,referral,33953
1,organic,9190
2,data deleted,514
3,cpc,15
4,none,0


## Replace values

- bad referral naming from shop.googlemerchandisestore.com means badly tracked and we should consider direct traffic instead
- medium =none is referring to direct traffic and we will use the same name to not confuse with null values
- data deleted is most likely paid campaign by google to avoid confidential data exposure so we will replace with cpc / google as well


In [45]:

# data.loc[(data['traffic_source_source'] == 'shop.googlemerchandisestore.com') & (data['traffic_source_medium'] == 'referral'), 'traffic_source_source'] = 'direct'
# data.loc[(data['traffic_source_source'] == 'shop.googlemerchandisestore.com') & (data['traffic_source_medium'] == 'referral'), 'traffic_source_medium'] = 'direct'
# source is merchandisestore.com then direct (bad parameter)
data.loc[(data['traffic_source_source'] == 'shop.googlemerchandisestore.com') & (data['traffic_source_medium'] == 'referral'), ['traffic_source_medium', 'traffic_source_source']] = ['direct','direct']


# referral traffic
data.loc[(data['traffic_source_medium'] == 'referral'), ['traffic_source_medium', 'traffic_source_source']] = ['referral','referral_link']

# google organic
data.loc[(data['traffic_source_source'].isnull()) & (data['traffic_source_medium']== 'organic'),  ['traffic_source_medium', 'traffic_source_source']] = ['organic','google']


# when we have direct traffic it is direct traffic
data.loc[(data['traffic_source_source'] == 'direct'), 'traffic_source_medium'] = 'direct'

# data deleted is paid campaign cpc by google
# data.loc[(data['traffic_source_source'] == 'data deleted') | (data['traffic_source_medium'] == 'data deleted'), 'traffic_source_medium'] = 'cpc'
# data.loc[(data['traffic_source_source'] == 'data deleted') | (data['traffic_source_medium'] == 'data deleted'), 'traffic_source_source'] = 'google'
data.loc[(data['traffic_source_source'] == 'data deleted') | (data['traffic_source_medium'] == 'data deleted'), ['traffic_source_medium', 'traffic_source_source']] = ['cpc', 'google']


# full null values are direct
data.loc[(data['traffic_source_source'].isnull()) & (data['traffic_source_medium'].isnull()),  ['traffic_source_medium', 'traffic_source_source']] = ['direct','direct']
# data.loc[(data['traffic_source_source'].isnull()) & (data['traffic_source_medium'].isnull()), 'traffic_source_medium'] = 'direct'



# data = fill_nulls_based_on_top_value_multiple_columns(data, 'traffic_source_source', ['device_category','geo_country'])
# data = fill_nulls_based_on_top_value_multiple_columns(data, 'traffic_source_medium', ['device_category','geo_country','traffic_source_source','traffic_source_medium'])


data.groupby(['traffic_source_source','traffic_source_medium']).agg(
    unique_session_count=('session_id', 'nunique')
).sort_values('unique_session_count',ascending=False).reset_index()

,traffic_source_source,traffic_source_medium,unique_session_count
0,direct,direct,47839
1,google,organic,35482
2,google,cpc,11838
3,referral_link,referral,10156
4,shop.googlemerchandisestore.com,organic,2


# Page columns

In [46]:
df_page_ref=data.groupby(['page_location','page_referrer']).agg(
    unique_session_count=('session_id', 'nunique')
).sort_values('unique_session_count',ascending=False).reset_index()

In [47]:
# remove domain name for better readability
data['page_location'] = data['page_location'].str.replace(r'shop.googlemerchandisestore.com/store.html', 'shop.googlemerchandisestore.com/').str.strip()
data['page_location'] = data['page_location'].str.replace(r'+', ' ').str.strip()
data['page_location'] = data['page_location'].str.replace(r'https://', '').str.strip()
data['page_location'] = data['page_location'].str.replace(r'http://', '').str.strip()
data['page_location'] = data['page_location'].str.replace(r'www.', '').str.strip()


df_pages_agg=data.groupby(['page_location','page_title']).agg(
    unique_session_count=('session_id', 'nunique')
).sort_values('unique_session_count',ascending=False).reset_index()

df_pages_agg

,page_location,page_title,unique_session_count
0,shop.googlemerchandisestore.com/,Home,32340
1,shop.googlemerchandisestore.com/Google Redesig...,Men's / Unisex | Apparel | Google Merchandise ...,11688
2,googlemerchandisestore.com/,Google Online Store,11260
3,shop.googlemerchandisestore.com/Google Redesig...,Apparel | Google Merchandise Store,9989
4,shop.googlemerchandisestore.com/Google Redesig...,Sale | Google Merchandise Store,8953
...,...,...,...
1066,shop.googlemerchandisestore.com/google redesig...,Page Unavailable,1
1067,shop.googlemerchandisestore.com/google re,Page Unavailable,1
1068,shop.googlemerchandisestore.com/google campus,Page Unavailable,1
1069,shop.googlemerchandisestore.com/frequently-ask...,Page Unavailable,1


## split categories in page paths

In [48]:
# Step 1: Split 'page_location' into 4 parts (max)
split_columns = data['page_location'].str.split('/', n=4, expand=True)

# Step 2: Assign the first three parts to new columns (ignore the first empty part if there is a leading '/')
data['domain'] = split_columns[0] # url_domain 
data['page_path_1'] = split_columns[1].replace('', pd.NA)
data['page_path_2'] = split_columns[2].replace('', pd.NA)
data['page_path_3'] = split_columns[3].replace('', pd.NA)




df_pages_total=data[['page_title','page_location','page_path_1','page_path_2','page_path_3']]


## fill length page path with page title

In [49]:
data['page_path_1'] = data.apply(
    lambda row: row['page_title'] if pd.isna(row['page_path_2']) else row['page_path_1'], 
    axis=1
)

df_pages_total=data[['page_title','page_location','page_path_1','page_path_2','page_path_3']]


## fill the other page path levels with previous page path column

This will allow for hierarchical encoding without sacrificing columns or rows

In [50]:
# Fill 'page_path_2' with 'page_path_1' if 'page_path_2' is null
data['page_path_2'] = data.apply(
    lambda row: row['page_path_1'] if pd.isna(row['page_path_2']) else row['page_path_2'], 
    axis=1
)

# Fill 'page_path_3' with 'page_path_2' if 'page_path_3' is null
data['page_path_3'] = data.apply(
    lambda row: row['page_path_2'] if pd.isna(row['page_path_3']) else row['page_path_3'], 
    axis=1
)


df_pages_total=data[['page_title','page_location','page_path_1','page_path_2','page_path_3']]

In [51]:

df_pages_agg=data.groupby(['page_title','domain','page_location','page_path_1','page_path_2','page_path_3']).agg(
    unique_session_count=('session_id', 'nunique')
).sort_values('unique_session_count',ascending=False).reset_index()

df_pages_agg

,page_title,domain,page_location,page_path_1,page_path_2,page_path_3,unique_session_count
0,Home,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/,Home,Home,Home,32340
1,Men's / Unisex | Apparel | Google Merchandise ...,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/Google Redesig...,Google Redesign,Apparel,Mens,11688
2,Google Online Store,googlemerchandisestore.com,googlemerchandisestore.com/,Google Online Store,Google Online Store,Google Online Store,11260
3,Apparel | Google Merchandise Store,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/Google Redesig...,Google Redesign,Apparel,Apparel,9989
4,Sale | Google Merchandise Store,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/Google Redesig...,Google Redesign,Clearance,Clearance,8953
...,...,...,...,...,...,...,...
1049,Page Unavailable,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/google redesig...,google redesign,accessories,google mouse pad navy,1
1050,Page Unavailable,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/google redesig...,google redesign,accessories,google mouse pad navy,1
1051,Page Unavailable,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/google redesig...,google redesign,accessories,google maps pin,1
1052,Page Unavailable,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/google redesig...,google redesign,accessories,google lip balm sanitizer duo,1
